<a href="https://colab.research.google.com/github/drsimonturega/ir_spec_man/blob/main/pecsvtocsv_multi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Infrared multi-spectral manipulation: *.asp to *.csv
### This it the multi sprcta version
### You need to be logged into your google acount to use this code
### There are three sections of code that need to be run in sequence;
- ### Lab book code for this experiemnt
- ### Upload your .asp spectra files (.asp from a Agilent ATR-FTIR spectrometer)
- ### Extract data from *.asp to transposed *.csv

### This will provide a *.zip file for download containing your orginal files and your spectra as a transposed *.csv.

In [1]:
# @title ## Lab book code for this experiemnt
# @markdown ### Enter the lab book code for this experiemnt
lab_book_code = 'smt_test_001' # @param {type:"string"}
import os
import shutil

os.chdir("/content")
try:
    os.mkdir(lab_book_code)
except FileExistsError:
    print(f'A folder for {lab_book_code} already exists you may over write your\
        data if you continue')
    #the lines below remove the folder! not currently implimented
    #os.chdir("..")
    #shutil.rmtree(lab_book_code)
try:
    !pwd
    os.chdir(lab_book_code)
    !pwd
except FileNotFoundError:
    print(f'try again')



/content
/content/smt_test_001


## Upload your .asp spectra file (.asp from a Agilent ATR-FTIR spectrometer)



In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
spec_list = list(uploaded.keys())
print(f'{spec_list}')
#spec_trans(str)
for i in range(len(spec_list)):
    spec_list[i] = str(spec_list[i])
    spec_trans = spec_list[i]
    #print(f'spec_trans')
    spec_trans = spec_trans[:-4]
    spec_list[i] = spec_trans


Saving Administrator 04.csv to Administrator 04.csv
Saving Administrator 05.csv to Administrator 05.csv
Saving Administrator 06.csv to Administrator 06.csv
User uploaded file "Administrator 04.csv" with length 54389 bytes
User uploaded file "Administrator 05.csv" with length 54560 bytes
User uploaded file "Administrator 06.csv" with length 54888 bytes
['Administrator 04.csv', 'Administrator 05.csv', 'Administrator 06.csv']


In [32]:
print(f'{spec_list}')
import pandas as pd
spec_sht_name = spec_list
spec_inp = pd.read_csv(f'{spec_sht_name[0]}.csv')
spec_inp = pd.DataFrame.transpose(spec_inp)
spec_inp = spec_inp.drop(axis=1, columns=[0])
try:    #set index
            spec_inp = spec_inp.reset_index(drop=True)
            #build well name and remove related columns
            #spec_inp.iloc[0:,1] = spec_inp.iloc[0:,1].astype(str)
            #spec_inp.iloc[0:,2] = (spec_inp.iloc[0:,0].astype(str)
            #    + spec_inp.iloc[0:,1].astype(str))
            #spec_inp = spec_inp.drop(axis=1, columns=[4,5])
            #spec_inp = pd.DataFrame.drop(spec_inp,axis=0, index=0)
            #~spec_inp.iloc[0,0] = "Well"
            spec_inp.rename(columns= spec_inp.iloc[0,0:], inplace=True)
            spec_inp = spec_inp.reset_index(drop=True)
            spec_inp = pd.DataFrame.drop(spec_inp, axis=0, index=0)
            spec_inp = spec_inp.reset_index(drop=True)
            spec_inp.drop(spec_inp.tail(1).index,inplace=True)
except IndexError:
            print('Check the your input file!')
spec_inp.head()



['Administrator 04', 'Administrator 05', 'Administrator 06']


,4000.00,3999.00,3998.00,3997.00,3996.00,3995.00,3994.00,3993.00,3992.00,3991.00,...,659.00,658.00,657.00,656.00,655.00,654.00,653.00,652.00,651.00,650.00
0,0.0006,0.0006,0.0006,0.0006,0.0007,0.0007,0.0007,0.0006,0.0006,0.0005,...,0.1276,0.1276,0.1275,0.1273,0.1269,0.1267,0.1273,0.1281,0.1291,0.1299


### Extract data from *.asp to transposed *.csv

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from sklearn.linear_model import LinearRegression
import statistics
from tabulate import tabulate
#from google.colab import files
import shutil


class asptospectocsv():


    def __init__(self, asp_input):
        return

    def spec_import(self, spec_sht_name):
        #importing spectra from .asp from a waters? ATR-FTIR spectrometer
        spec_inp= pd.DataFrame()
        try:
            spec_inp = pd.read_csv(spec_sht_name + '.csv')
        except PermissionError:
            print('The file we are trying to open may be open in another \
                program, please close and try again!')
        print(spec_inp.head())
        return spec_inp

    def asptodfd(self, spec_inp, spec_sht_name):
      from numpy import log as ln

      ir_spec_raw = spec_inp

      # we need th col name as a float?
      col_head = list(ir_spec_raw.columns)
      num_spec_points = col_head[(0)]

      # build the x axis (wavenumbers) from the file format
      high_wavnum = ir_spec_raw.iloc[0,0] #highest wavenumber
      high_wavnum = float(high_wavnum)
      low_wavnum = ir_spec_raw.iloc[1,0] #lowest wavenumber
      low_wavnum = float(low_wavnum)
      num_spec_points = float(num_spec_points)
      spec_gap = ( high_wavnum - low_wavnum ) / num_spec_points #calaculate spectral gap
      wav_num = []
      wav_nums = np.arange(low_wavnum, high_wavnum, spec_gap)
      wav_num_rev=[]

      for i in reversed(np.arange(low_wavnum, high_wavnum, spec_gap)): #build the list of wave numbers
        wav_num_rev.append(i)
      #set first col in dataframe as x axis (wavenumber)
      spec_sin = pd.DataFrame()
      spec_sin['wave number / cm^-1'] = wav_num_rev

      #add y axis (transmitance) to dataframe
      spec_trans=[] # a list to transport transmitance data
      spec_trans = ir_spec_raw.iloc[5:,0] #it gets sliced
      spec_trans = spec_trans.reset_index(drop=True) #it gets reindexed
      spec_sin['spec1'] = spec_trans #spec name
      bg = pd.DataFrame.transpose(spec_sin)

      bg.rename(columns=bg.iloc[0,0:], inplace=True) # set row 0 as col titles
      bg = bg.reset_index(drop=True)
      bg = pd.DataFrame.drop(bg, axis=0, index=0)
      #print(bg)
      spec_sin = bg
      spec_sin.insert(loc = 0, column = 'sample ID', value = spec_sht_name)
      spec_sin = spec_sin.reset_index(drop=True) #it gets reindexed
      #print(spec_sin)
      return spec_sin

    def asptodfd_multi(self,  spec_list):
      spec_new = pd.DataFrame()
      print(f'{len(spec_list)}')
      print(f'{(spec_list)}')
      spec_n = asptospectocsv.spec_import(self, spec_list[0])
      spec_new = asptospectocsv.asptodfd(self, spec_n, spec_list[0])
      for i in range(1,(len(spec_list))):
        print(f'{(spec_list[i])}')
        spec_n = asptospectocsv.spec_import(self, spec_list[i])
        spec_trans = asptospectocsv.asptodfd(self, spec_n, spec_list[i])
        print(spec_trans)
        #print(spec_new)
        spec_new.loc[i] = spec_trans.loc[0]
      #print(spec_trans)
      print(spec_new)

      return spec_new


    def savendl(self,spec_new):
      print(f'what is this {spec_new}')
      try: #output results
            spec_new.to_csv("ir_" + lab_book_code + ".csv",index=False)
      except PermissionError:
            print('The file we are trying to open may be open in another \
                program, please close and try again!')
      os.chdir("/content")
      shutil.make_archive(lab_book_code, 'zip', lab_book_code)
      files.download(f'{lab_book_code}.zip')
      return

#specasp = asptospectocsv(spec_list)
#loop_csv = asptospectocsv.asptodfd_multi(specasp, spec_list)
#asptospectocsv.savendl(specasp, loop_csv)




5
['BAS_25_001_2023-06-14T09-55-56 (1)', 'BAS_100_001_2023-06-14T09-58-46 (1)', 'BAS_Buffer_001_2023-06-13T15-54-39 (1)', 'BAS_Buffer_001_2023-06-13T15-56-35 (1)', 'BAS_Water_2023-06-13T15-51-57 (1)']
BAS_100_001_2023-06-14T09-58-46 (1)
                             sample ID  3995.7100160365744  \
0  BAS_100_001_2023-06-14T09-58-46 (1)           99.648306   

   3991.9865327369325  3988.26304943729  3984.5395661376483  \
0           99.539681         99.469755           99.494827   

   3980.8160828380064  3977.0925995383645  3973.3691162387227  \
0           99.627842           99.610026           99.683539   

   3969.6456329390808  3965.922149639439  ...  432.3364982792365  \
0           99.725783          99.546103  ...          30.636227   

   428.61301497959454  424.8895316799526  421.16604838031066  \
0           30.474561          30.239649           31.839339   

   417.4425650806687  413.7190817810268  409.99559848138483  \
0          33.023719           33.74774           3

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>